In [1]:
from faker import Faker
import numpy as np 
import pandas as pd

# importación de librerías propias
import utils.folders_tb as fold 
import utils.tools as tools
import utils.sql_tb as sql
from utils.mysql_driver import MySQL
import utils.mysql_driver as drv

fake = Faker('es_ES')
df_localidades = fold.read_Loc_SP()
df_localidades.drop(columns='Unnamed: 0', inplace=True)


In [2]:

df_localidades

,CA,provincia,poblacion,CP,lat,lon
0,Cataluña,Barcelona,Barcelona,8001,"41,38424664","2,17634927"
1,Cataluña,Barcelona,Barcelona,8002,"41,38424664","2,17634927"
2,Cataluña,Barcelona,Barcelona,8003,"41,38424664","2,17634927"
3,Cataluña,Barcelona,Barcelona,8004,"41,38424664","2,17634927"
4,Cataluña,Barcelona,Barcelona,8005,"41,38424664","2,17634927"
...,...,...,...,...,...,...
319,Castilla la Mancha,Toledo,Toledo,45004,"39,85715187","-4,02431421"
320,Castilla la Mancha,Toledo,Toledo,45005,"39,85715187","-4,02431421"
321,Castilla la Mancha,Toledo,Toledo,45006,"39,85715187","-4,02431421"
322,Castilla la Mancha,Toledo,Toledo,45007,"39,85715187","-4,02431421"


In [6]:
df_localidades['CP'] = df_localidades['codigopostalid'].apply(lambda x: str(x) if x > 10000 else '0'+str(x)) #se convierte el CP original en string 
df_localidades.drop(columns='codigopostalid', inplace=True)
df_localidades['CA'] = df_localidades['CP'].apply(lambda x: tools.CP_toCom(x))
df_localidades = df_localidades[['CA','provincia','poblacion','CP','lat','lon']]
fold.save_df_to_csv(df_localidades,'final_loc')


In [13]:
df_localidades['CP'][25]

8026

In [2]:
customers_list = []
discap_list = ['motora','visual','auditiva','cognitiva','orgánica','ninguna']
columnas_BDD = ['perfil','nombre','apellidos','email','password','token','dni','direc','CP','ciudad','provincia','CA', 'lat','lon','dicapacidad','grado', 'fake_val']

for _ in range(1000):

    user_fields = []
    # Generación del perfil, nombre y appelidos
    user_fields.append('usuario')
    user_fields.append(fake.first_name())
    apellidos = fake.last_name() + ' ' + fake.last_name()
    user_fields.append(apellidos)

    # Generación datos de alta
    user_fields.append(fake.email())
    user_fields.append(fake.password(digits=True, upper_case=True, lower_case=True, length=10))
    user_fields.append(fake.windows_platform_token())
    user_fields.append(fake.nif())
    
    # Generación de los campos de la dirección
    address_seed = fake.address()
    address_seed_list = list(address_seed.split())
    address_CP = address_seed_list[-1]
    address_city = address_seed_list[-2].replace(',', '')
    address_clean = address_seed.replace(address_CP, '')
    address_clean = address_clean.replace(address_city, '')
    address_clean = address_clean.replace('\n', '')
    address_clean = address_clean.replace(',', '')
    user_fields.append(address_clean)

    # Generador aleatorio de ubicación
    rd_seed = np.random.randint (0,324)
    user_fields.append(df_localidades['CP'][rd_seed])
    user_fields.append(df_localidades['poblacion'][rd_seed])
    user_fields.append(df_localidades['provincia'][rd_seed])
    user_fields.append(df_localidades['CA'][rd_seed])
    user_fields.append(df_localidades['lat'][rd_seed])
    user_fields.append(df_localidades['lon'][rd_seed])

    # Generación de la discapacidad y porcentaje
    discap = discap_list[np.random.randint(0,6)]
    user_fields.append(discap)
    if discap != 'ninguna':
        porcen_disc = np.random.randint(1,11)*3 + np.random.randint(1,21)+ np.random.randint(1,51)
        user_fields.append(porcen_disc)
    else:
        rand_usu = np.random.randint(1,101)
        porcen_disc = 0
        if rand_usu >= 95:
            user_fields[0] = 'certificador'
        else:
            user_fields.append(porcen_disc)

    # generación de valoraciones
    if porcen_disc > 70:
        val = np.random.randint(1,3)*3 + np.random.randint(1,5)+ np.random.randint(1,20)
    elif porcen_disc > 30:
        val = np.random.randint(0,3)*2 + np.random.randint(1,5)
    else:
        val = np.random.randint(0,3) + np.random.randint(0,5)
    user_fields.append(val)

    # Integración del usuario generado en la lista de usuarios
    customers_list.append(user_fields)


df_users = pd.DataFrame(customers_list, columns= columnas_BDD)
#df_users = pd.DataFrame(customers_list, columns= ['nombre','apellido 1','apellido 2','direc','CP','ciudad'])
df_users

,perfil,nombre,apellidos,email,password,token,dni,direc,CP,ciudad,provincia,CA,lat,lon,dicapacidad,grado,fake_val
0,usuario,Concepción,Mulet Escrivá,saezjacobo@canton.com,(sX+#Ne5$3,Windows 98,01981645B,Vial Rosalinda Gimeno 18,33697,Gijón,Asturias,Principado de Asturias,"43,53920222","-5,65949323",auditiva,71,20.0
1,usuario,Rosalva,Ojeda Sobrino,fonsecaclara@bueno-baro.net,*$6B@Dd11%,Windows NT 5.01,93147581L,Rambla de Cándida Pellicer 35 Apt. 46,40195,Segovia,Segovia,Castilla y León,"40,9498703","-4,12524116",ninguna,0,0.0
2,usuario,Adrián,Salvador Batlle,albina22@gmail.com,4%r2NXe6&$,Windows NT 5.1,54332183L,Vial Chema Baró 38,28042,Madrid,Madrid,Comunidad de Madrid,"40,40841191","-3,68760088",motora,24,4.0
3,usuario,Baldomero,Ariza Herrera,vsarabia@yahoo.com,3#Vu7ZXlM5,Windows NT 6.1,33566162P,Pasadizo de Teodora Borja 378,8039,Barcelona,Barcelona,Cataluña,"41,38424664","2,17634927",auditiva,45,4.0
4,usuario,Aurelia,Cabeza Morata,alegrecandida@hotmail.com,*I0!6Drsxd,Windows NT 5.0,75803894B,Alameda Lorena Sosa 17 Apt. 30 Santa Cruz de,33299,Gijón,Asturias,Principado de Asturias,"43,53920222","-5,65949323",orgánica,73,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,usuario,Damián,Tenorio Montes,juanitofigueroa@gmail.com,_sA0YViuhv,Windows 95,54850107Y,Cuesta Rosa Arnaiz 494 Puerta 0,23002,Jaén,Jaén,Andalucía,"37,7651913","-3,7903594",orgánica,41,6.0
996,usuario,Paloma,Milla Cisneros,lluchdorita@gmail.com,M8DmzLGY@W,Windows NT 4.0,62458510D,Pasadizo Pablo Planas 39 Puerta 9,17005,Girona,Girona,Cataluña,"41,98186075","2,82411899",orgánica,59,3.0
997,usuario,Judith,Plaza Pellicer,rochajesus@aliaga-matas.es,(121ScvQ71,Windows NT 5.2,26122534P,Pasadizo Nieves Quero 82,28002,Madrid,Madrid,Comunidad de Madrid,"40,40841191","-3,68760088",motora,66,5.0
998,usuario,Jessica,Palomino Muñoz,augustovelazquez@marin-bello.com,A&Xo4XrWSy,Windows NT 10.0,14092013M,Alameda de Rosa Carbó 422,36153,Pontevedra,Pontevedra,Galicia,"42,43158576","-8,643672092",orgánica,56,4.0


In [5]:
fold.save_df_to_csv(df_users,'usuarios')


In [6]:
sql.insert_sqlList (df_users, 'usuarios')

Connected to MySQL server [TurismoAccesible]
Executed 

DROP TABLE IF EXISTS usuarios

 successfully
Executed 

CREATE TABLE usuarios(
        ID INT(11) NOT NULL AUTO_INCREMENT,
        MOMENTO TIMESTAMP NOT NULL,
        PERFIL VARCHAR(255) NOT NULL,NOMBRE VARCHAR(255) NOT NULL,APELLIDOS VARCHAR(255) NOT NULL,EMAIL VARCHAR(255) NOT NULL,PASSWORD VARCHAR(255) NOT NULL,TOKEN VARCHAR(255) NOT NULL,DNI VARCHAR(255) NOT NULL,DIREC VARCHAR(255) NOT NULL,CP INT NOT NULL,CIUDAD VARCHAR(255) NOT NULL,PROVINCIA VARCHAR(255) NOT NULL,CA VARCHAR(255) NOT NULL,LAT VARCHAR(255) NOT NULL,LON VARCHAR(255) NOT NULL,DICAPACIDAD VARCHAR(255) NOT NULL,GRADO INT NOT NULL,FAKE_VAL FLOAT(24) NOT NULL,
        PRIMARY KEY (ID))

 successfully
Executed 

INSERT INTO usuarios 
        (MOMENTO,PERFIL,NOMBRE,APELLIDOS,EMAIL,PASSWORD,TOKEN,DNI,DIREC,CP,CIUDAD,PROVINCIA,CA,LAT,LON,DICAPACIDAD,GRADO,FAKE_VAL)
                    VALUES 
        (NOW(), 'usuario', 'Concepción', 'Mulet Escrivá', 'saezjacobo@canton.